In [2]:
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras.models import Sequential

In [2]:
class LSTMClassifier():

    def __init__(self, input_dim, input_len, mode=None, out_dim=50, hidden_out_dim=256, dropout=0.0, verbose=1):

        self.verbose = verbose

        self.model = Sequential()
        
        if mode is None:
            print('Embeded-LSTM model...')
            self.model.add(Embedding(input_dim=input_dim,
                                     output_dim=out_dim, input_length=input_len))
            self.model.add(
                LSTM(activation='relu', return_sequences=True, units=hidden_out_dim))
            self.model.add(Dropout(dropout))
            self.model.add(LSTM(activation='relu', units=hidden_out_dim,
                                recurrent_activation='hard_sigmoid'))
            self.model.add(Dropout(dropout))
            
        elif mode == 'low-dim':
            print('Dense model...')
            self.model.add(
                Dense(output_dim=out_dim, input_shape=(input_len, ), activation='sigmoid'))
            self.model.add(Dropout(dropout))
            self.model.add(Dense(output_dim=out_dim, activation='sigmoid'))
            self.model.add(Dropout(dropout))
            self.model.add(Dense(output_dim=out_dim, activation='sigmoid'))
            
        self.model.add(Dense(1, activation='sigmoid'))

        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam', metrics=['accuracy'])
        print(self.model.summary())

    def fit(self, X, Y, epochs=3, batch_size=10, validation_split=0.1):

        return self.model.fit(X, Y, batch_size=batch_size,
                              epochs=epochs, verbose=self.verbose,
                              callbacks=[TerminateOnNaN(), EarlyStopping(monitor='loss')])

    def evaluate(self, x, y, batch_size=10):
        score, acc = self.model.evaluate(
            x, y, batch_size=batch_size, verbose=self.verbose)
        return score, acc

    def predict(self, x, batch_size=10):
        return self.model.predict(x, batch_size=batch_size, verbose=self.verbose)

Using TensorFlow backend.


In [ ]:
def LSTM_model(X_matrix, Y, x_matrix, y, vocab_len, feature_len, epochs = 1, batch_size=100, mode = None, plot=False, filename = None, verbose=1):

    LSTM_model = LSTMClassifier(vocab_len + 1, feature_len, mode=mode, verbose=verbose)

    LSTM_model.fit(X_matrix, Y, epochs=epochs, batch_size=batch_size, validation_split=None)
    
    if plot:
        file = '{}/{}-boundary.png'.format(result_folder, filename)
        plot_boundary(train_tsne_seq_matrix, Y, LSTM_model, filename = file, h=1)

    print('\n----------------------------Parameters--------------------------------\n')
#     train_score, train_acc = LSTM_model.evaluate(train_LSTM_seq_matrix, Y, batch_size=batch_size)
#     print('Training score : ', train_score, '  Training Acurracy : ', train_acc)
#     print('\n------------------------------------------------------------------------------------\n')

    y_trainig_hat = LSTM_model.predict(X_matrix)
    y_trainig_hat = np.where(y_trainig_hat > 0.5, 1, 0)
#     print('Training precision score ', precision_score(Y, y_trainig_hat))
#     print('\n------------------------------------------------------------------------------------\n')
#     print('Training f1 score ', f1_score(Y, y_trainig_hat))
#     print('\n------------------------------------------------------------------------------------\n')

    train_metric = metric_score(Y, y_trainig_hat.reshape(len(y_trainig_hat)), type='Training')
    
#     test_score, test_acc = LSTM_model.evaluate(test_LSTM_seq_matrix, y, batch_size=batch_size)
#     print('Test score : ', test_score, '  Test Acurracy : ', test_acc)
#     print('\n------------------------------------------------------------------------------------\n')

    y_test_hat = LSTM_model.predict(x_matrix)
    y_test_hat = np.where(y_test_hat > 0.5, 1, 0)
#     print('Test precision score ', precision_score(y, y_test_hat))
#     print('\n------------------------------------------------------------------------------------\n')
#     print('Test f1 score ', f1_score(y, y_test_hat))
#     print('\n------------------------------------------------------------------------------------\n')
          
    test_metric = metric_score(y, y_test_hat.reshape(len(y_test_hat)), type='Test')
    print(LSTM_model.model)
    if filename is not None:
        pd_best = pd.DataFrame({"best_params" : {'model': 'epochs {} , batch_size  {}'.format(epochs, batch_size), 'model__alpha': batch_size}})
        
        save_results_html(filename, pd.concat([pd.concat([train_metric, test_metric], axis=1, sort=False), pd_best], keys=['Score', 'Model'], axis=1, sort=False))